In [ ]:
# Importing libraries
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core import display as ICD

In [ ]:
# Supress some warnings for better clarity
# Set some options also
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.plotting.backend = "plotly"
pd.set_option('mode.chained_assignment', None)
pd.reset_option("mode.chained_assignment")

In [ ]:
# Variable to determine if items should be plotted
# Warning: if true, the notebook may run out of memory due to the plots
# Only set if interested in plotting
plot_timeseries = False
plot_importances = False

# Pre-Processing

In [ ]:
# Create a dictionary in order to keep all the datasets in one place
datasets = {}

In [ ]:
# create a dataframe for sales
sales_df = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv',
                          dtype={
                            'store_nbr': 'category',
                            'family': 'category',
                            'onpromotion': 'uint32'
                            },
                         parse_dates=['date'],
                         date_parser= lambda x: pd.to_datetime(x, format='%Y-%m-%d').to_period('D'),
                         infer_datetime_format=True,
                         index_col=['date', 'store_nbr', 'family'],
                         usecols = ['date', 'store_nbr', 'family','onpromotion', 'sales'])
datasets['sales_init'] = sales_df
datasets['sales_init'].head(5)

In [ ]:
# create a dataframe for test
test_df = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv',
                          dtype={
                            'store_nbr': 'category',
                            'family': 'category',
                            'onpromotion': 'uint32'
                            },
                         parse_dates=['date'],
                         date_parser= lambda x: pd.to_datetime(x, format='%Y-%m-%d').to_period('D'),
                         infer_datetime_format=True,
                         index_col=['date', 'store_nbr', 'family'],
                         usecols = ['date', 'store_nbr', 'family','onpromotion'])
datasets['test_init'] = test_df
datasets['test_init'].head(5)

In [ ]:
# create a dataframe for holidays
holidays_df = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv',
                           dtype={
                             # Used str insted of category
                             'type': 'str',
                             'locale': 'str',
                             'locale_name': 'str',
                             'transferred': 'boolean'
                             },
                          parse_dates=['date'],
                          date_parser= lambda x: pd.to_datetime(x, format='%Y-%m-%d').to_period('D'),
                          infer_datetime_format=True,
                          index_col=['date'],
                          usecols = ['date', 'type', 'locale','locale_name','transferred'])
datasets['holidays_init'] = holidays_df
datasets['holidays_init'].head(5)

In [ ]:
# create a dataframe for oil
oil_df = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv',
                     parse_dates=['date'],
                     date_parser= lambda x: pd.to_datetime(x, format='%Y-%m-%d').to_period('D'),
                     infer_datetime_format=True,
                     index_col=['date'])
datasets['oil_init'] = oil_df
datasets['oil_init'].head(5)

In [ ]:
# create a dataframe for transactions
transactions_df = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv',
                          dtype={
                            'store_nbr': 'category',
                            'transactions': 'int'
                            },
                         parse_dates=['date'],
                         date_parser= lambda x: pd.to_datetime(x, format='%Y-%m-%d').to_period('D'),
                         infer_datetime_format=True,
                         index_col=['date', 'store_nbr'],
                         usecols = ['date', 'store_nbr', 'transactions'])
datasets['transactions_init'] = transactions_df
datasets['transactions_init'].head(5)

In [ ]:
# create a dataframe for stores
stores_df = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv',
                          dtype={
                            'store_nbr': 'category',
                            'city':  'category',
                            'state':  'category',
                            'cluster':  'category',
                            },
                         usecols = ['store_nbr', 'city', 'state', 'cluster', 'type'])
datasets['stores_init'] = stores_df
datasets['stores_init'].head(5)

In [ ]:
# find the size of all dataframes combined
all_df_size_mb = sum([sys.getsizeof(df) for df in datasets.values()]) / 10**6
print(f'All dataframes reserve {all_df_size_mb} MB')

In [ ]:
# method to preview dataframe with better clarity
def dataset_preview(df, head=3):
    print('='*50)
    print('Preview:')
    ICD.display(df.head(head))
    print('*'*50)
    print('Describe:')
    ICD.display(df.describe())
    print('*'*50)
    print('Info:')
    print(df.info())
    print('='*50)
    
# method to create lag features
def create_lags(df, target_col, lags=1, suffix='_lag', drop_target=True):
    for i in range(1, lags + 1) :
        df[f'{target_col}({suffix}{i})'] = df[target_col].shift(i)
    if drop_target:
        df = df.drop(columns=[target_col])
    return df

In [ ]:
# cherry-picking stores based on some peculiarities and adding new corresponding features
stores_df = datasets['stores_init'].copy()
# add features based on Stores origins
#  - areas with only one store
stores_df['uniquestore'] = stores_df['city'].apply(lambda x: 0 if x in ['Quito', 'Guayaquil', 'Santo Domingo', 'Cuenca', 'Manta', 'Machala', 'Latacunga', 'Ambato'] else 1)
# - stores which were inaugurated in train/test period
stores_df['newstore'] = stores_df['store_nbr'].apply(lambda x: 1 if x in [19, 20, 21, 28, 35, 41, 51, 52] else 0)
stores_df = stores_df.rename(columns={'type' : 'store'}) 
stores_df = stores_df.set_index('store_nbr')

In [ ]:
# add the dataframe with new Stores features to the dictionary
datasets['stores_trans'] = stores_df

In [ ]:
# preview the holidays dataset
dataset_preview(datasets['holidays_init'])

In [ ]:
# create a series object for range of dates (training data + test data) from 2013-01-01 to 2017-08-31
total_days_range = pd.Series(pd.date_range('2013-01-01', '2017-08-31').to_period('D'), name='date')

# create a dataframe out of the series object
calendar_df = pd.DataFrame(index=total_days_range)

# merge the calendar dataframe with the holidays dataset
calendar_df = calendar_df.merge(datasets['holidays_init'], how='left',on=['date'])
calendar_df.head()

In [ ]:
# map the missing rows as follows: 'type' - 'Work Day', 'locale' - 'National', 'locale_name' - 'Ecuador', 'transferred' - False
calendar_df.loc[calendar_df.isna().any(axis=1), ['type', 'locale', 'locale_name', 'transferred']] = 'Work Day', 'National', 'Ecuador', False
calendar_df.head()

In [ ]:
# apply corrections based on holidays

# different holiday types are:
# * Work Day
# * Bridge
# * Transfer
# * Event
# * Holiday
# * Additional

# initialize all the days as working
# * if the day is a working day, set wd = True
# * if the day is not a working day, set wd = False

easter_dates = ['2017-04-16', '2016-03-27', '2015-04-05', '2014-04-20', '2013-03-31']
easter_dates = [pd.to_datetime(date, format='%Y-%m-%d').to_period('D') for date in easter_dates]

calendar_df['wd'] = True
calendar_df.loc[calendar_df.type == 'Bridge'  , 'wd'] = False
calendar_df.loc[calendar_df.type == 'Transfer', 'wd'] = False
calendar_df.loc[(calendar_df.type == 'Additional') & (calendar_df.transferred == False), 'wd'] = False
calendar_df.loc[(calendar_df.type == 'Holiday') & (calendar_df.transferred == False), 'wd'] = False
calendar_df.loc[calendar_df.index.get_level_values('date').isin(easter_dates), 'wd'] = False

calendar_df['isevent'] = False
calendar_df.loc[calendar_df.type == 'Event'  , 'isevent'] = True
calendar_df.loc[calendar_df.index.get_level_values('date').isin(easter_dates), 'isevent'] = True

calendar_df = pd.get_dummies(calendar_df, columns=['type'])
calendar_df = pd.get_dummies(calendar_df, columns=['locale'])

calendar_df.drop(columns=['locale_name', 'transferred'], inplace = True) 

# aggregate duplicate/redundant rows
calendar_df = calendar_df.groupby('date').agg(lambda x: np.bitwise_or.reduce(x.values))

calendar_df.head()

In [ ]:
# add the calendar dataframe to the dictionary
datasets['calendar'] = calendar_df

In [ ]:
# preview oil dataset
dataset_preview(datasets['oil_init'])

In [ ]:
oil_df = datasets['oil_init'].copy()

# interpolate missing values
oil_df = oil_df.resample('D').mean().interpolate(limit_direction='backward').reset_index()

# add 4-day lags
oil_df = create_lags(oil_df, target_col='dcoilwtico', lags=4, suffix='_lag', drop_target=False)
# add weekly averaged values as a new feature
oil_df['oil_week_avg'] = oil_df['dcoilwtico'].rolling(7).mean()
# add biweekly averaged values as a new feature
oil_df['oil_biweek_avg'] = oil_df['dcoilwtico'].rolling(14).mean()
# add monthly averaged values as a new feature
oil_df['oil_1_month_avg'] = oil_df['dcoilwtico'].rolling(30).mean()
# add bimonthly averaged values as a new feature
oil_df['oil_2_month_avg'] = oil_df['dcoilwtico'].rolling(60).mean()

# drop NaN rows
oil_df.dropna(inplace = True)

# set index to date
oil_df = oil_df.set_index('date')
oil_df.head()

In [ ]:
# add the oil dataframe to the dictionary
datasets['oil_trans'] = oil_df

In [ ]:
# create a feature dataframe
feature_df = datasets['sales_init'].copy()
test_df = datasets['test_init'].copy()
test_df['sales'] = np.nan

feature_df = pd.concat([feature_df, test_df], axis=0)
feature_df.head()

In [ ]:
# add store features to this dataframe
feature_df = feature_df.merge(datasets['stores_trans'],  how='left',  left_index=True,  right_index=True)
feature_df.drop(feature_df.filter(regex='_y$').columns, axis=1, inplace=True) 

# add a separate feature for stores that are closed
feature_df['isclosed'] = feature_df.groupby(by=['date', 'store_nbr'])['sales'].transform(lambda x: 1 if x.sum()==0 else 0)
feature_df.loc[((feature_df.index.get_level_values('date').year==2017) & (feature_df.index.get_level_values('date').month==8) & (feature_df.index.get_level_values('date').day>=16)) , 'isclosed'] = feature_df['isclosed'].apply(lambda x: 0) 

# merge calendar feature with this dataframe
feature_df = feature_df.merge(datasets['calendar'], how='left',  left_index=True, right_index=True, suffixes=('', '_y'))

# drop common columns
feature_df.drop(feature_df.filter(regex='_y$').columns, axis=1, inplace=True)

# add oil features to this dataframe
# lags and 7-days moving average
feature_df = feature_df.merge(datasets['oil_trans'], how='left',  left_index=True, right_index=True)
feature_df.head()

# Visualization

In [ ]:
# Plot a yearly, monthly, and weekly plot for each family averaged over stores
# Provides insight into some time related features
# Create a dataframe of average sales per family per day, with year, day, and week info
if plot_timeseries:
    avg_family_df = datasets['sales_init'].copy()
    avg_family_df = avg_family_df.reset_index().groupby(['date', 'family']).sales.mean()
    avg_family_df = avg_family_df.reset_index().set_index('date')
    avg_family_df['year'] = avg_family_df.index.year
    avg_family_df['dayofyear'] = avg_family_df.index.dayofyear
    avg_family_df['month'] = avg_family_df.index.month
    avg_family_df['dayofmonth'] = avg_family_df.index.day
    avg_family_df['week'] = avg_family_df.index.week
    avg_family_df['dayofweek'] = avg_family_df.index.dayofweek

In [ ]:
# Plot average sales per family per year
if plot_timeseries:
    for family in avg_family_df.family.unique():
        for year in avg_family_df.year.unique():
            data = avg_family_df.loc[(avg_family_df.family == family) &
                                    (avg_family_df.year == year), 'sales']
            plt.plot(range(0, len(data)), data, label=year)
        plt.legend()
        plt.suptitle(family)
        plt.show()

In [ ]:
# Plot the sales information over the months
if plot_timeseries:
    over_the_months_df = avg_family_df.groupby(['month', 'dayofmonth', 'family']).sales.mean().reset_index()
    for family in over_the_months_df.family.unique():
        for month in over_the_months_df.month.unique():
            data = over_the_months_df.loc[(over_the_months_df.month == month) &
                                         (over_the_months_df.family == family), 'sales']
            plt.plot(range(0, len(data)), data, label=month)
        plt.legend()
        plt.suptitle(family)
        plt.show()

In [ ]:
# Plot the sales over each week of the month
if plot_timeseries:
    weekly_df = avg_family_df.groupby(['week', 'dayofweek', 'family']).sales.mean().reset_index()
    for family in weekly_df.family.unique():
        for week in weekly_df.week.unique():
            data = weekly_df.loc[(weekly_df.week == week) & (weekly_df.family == family), 'sales']
            plt.plot(range(0, len(data)), data, label=week)
        plt.suptitle(family)
        plt.show()

In [ ]:
# add time-related features

# * add yearly 
feature_df['year'] = feature_df.index.get_level_values('date').year.astype('int')
# * add quarterly trend
feature_df['quarter'] = feature_df.index.get_level_values('date').quarter.astype('int')
# * add daily trend
feature_df['day'] = feature_df.index.get_level_values('date').day.astype('int')

# * add weekly trend
feature_df['weekofyear'] = feature_df.index.get_level_values('date').week.astype('int')
# * add day of the week trend
feature_df['dayofweek'] = feature_df.index.get_level_values('date').day_of_week.astype('int')
# * add weekend trend
feature_df['isweekend'] = feature_df.dayofweek.apply(lambda x: 1 if x in (5,6) else 0)

# * add newyear day trend
feature_df['newyear'] = feature_df.index.get_level_values('date').dayofyear == 1
# * add school starting day trend
feature_df['startschool'] = feature_df.index.get_level_values('date').month.isin((4,5,8,9))
# * add weekend trend
feature_df['weekend'] = feature_df.index.get_level_values('date').dayofweek.isin((5,6))

# as wages in the public sector are paid every two weeks on the 15th and the last day of the month, add a trend for that too!
feature_df['wageday'] = ((feature_df.index.get_level_values('date').to_timestamp().is_month_end) | (feature_df.index.get_level_values('date').day == 15))

# add a trend for the days affected by earthquakes
earthquake_affected_dates = [d.to_period('D') for d in pd.date_range('2016-04-16', periods=20, freq='D').tolist()]
feature_df['earthquake_Aprin2016_effect'] = feature_df.index.get_level_values('date').isin(earthquake_affected_dates)

# add some dummy features
feature_df = pd.get_dummies(feature_df, columns=['year'], drop_first=True)
feature_df = pd.get_dummies(feature_df, columns=['quarter'], drop_first=True)
feature_df = pd.get_dummies(feature_df, columns=['dayofweek'], drop_first=True)

# add trend for weekly on promotion
feature_df['onpromotion_week_avg'] = feature_df['onpromotion'].rolling(7).mean()
# add trend for biweekly on promotion
feature_df['onpromotion_biweek_avg'] = feature_df['onpromotion'].rolling(14).mean()
# add trend for monthly on promotion
feature_df['onpromotion_1_month_avg'] = feature_df['onpromotion'].rolling(30).mean()
# add trend for bimonthly on promotion
feature_df['onpromotion_2_month_avg'] = feature_df['onpromotion'].rolling(60).mean()
# drop columns 'city', 'cluster', 'state', 'store'
feature_df.drop(columns=['city', 'cluster', 'state', 'store'], inplace=True)

In [ ]:
# Plot periodograms of the data to look at seasonality in the data
# Code from Kaggle Timeseries Course: https://www.kaggle.com/code/ryanholbrook/seasonality
def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    frequencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(frequencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=90,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

In [ ]:
# Plot the periodograms per family
if plot_timeseries:
    for family in avg_family_df.family.unique():
        ax = plot_periodogram(avg_family_df.loc[avg_family_df.family==family, 'sales'])
        plt.suptitle(family)
        plt.show()

In [ ]:
# Look at lag information in the dataset
if plot_timeseries:
    from statsmodels.graphics.tsaplots import plot_pacf
    for family in avg_family_df.family.unique():
        plot_pacf(avg_family_df.loc[avg_family_df.family==family, 'sales'], lags=14, title=family)

# Training

In [ ]:
# add deterministic time-series features
from statsmodels.tsa.deterministic import DeterministicProcess
from statsmodels.tsa.deterministic import CalendarFourier

# calculate fourier transforms in different frequency domains
fourierA = CalendarFourier(freq='A', order=5)
fourierQ = CalendarFourier(freq='Q', order=3) # added quarterly frequency
fourierM = CalendarFourier(freq='M', order=2)
fourierW = CalendarFourier(freq='W', order=3)
fourierB = CalendarFourier(freq='B', order=2) # added business day frequency

# instance of DeterministicProcess
dp = DeterministicProcess(index=feature_df.index.get_level_values('date'),
                          order=1,
                          seasonal=True,
                          constant=False,
                          additional_terms=[fourierA, fourierQ, fourierM, fourierW, fourierB],
                          drop=True)

dp_df = dp.in_sample()
# concatenate these deterministic features with feature dataframe
feature_df = pd.concat([feature_df.reset_index(level=['store_nbr', 'family']), dp_df], axis=1)

# set the indices again
feature_df = feature_df.set_index(['store_nbr', 'family'], append=True)

In [ ]:
# helper methods that will be used later-on in the code
import time
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_log_error


def _scorer(y_true, y_predicted):
    y_true = np.exp(y_true)-1
    y_predicted = np.exp(y_predicted)-1
    return mean_squared_log_error(y_true, y_predicted, squared=False)

scorer = make_scorer(_scorer, greater_is_better=False)

def index_timeseries_train_test_split(df, date_index=False, date_format='%Y-%m-%d', earliest_train_date=None, split_date=None, last_test_date=None, verbose=True):
    """
    =======
    Example
    =======
    * earliest_train_date: 2020-02-01
    * split_date: 2020-04-01
    * last_test_date: 2020-07-01

    Timeline:
    ---------------------------------------------------------------------------------------------------
    dates --> 2020-01-01 ... 2020-02-01 ... 2020-03-01 ... 2020-04-01 .. 2020-07-01 ... 2020-10-01 ...
                                 /\                          /\             /\
                                 |                           |              |
                        earliest_train_date             split_date     last_test_date
    ---------------------------------------------------------------------------------------------------

    Finallly:
    ---------
    2020-02-01 <= TRAIN DATASET DATES < 2020-04-01
    2020-04-01 <= TEST DATASET DATES < 2020-07-01                              
    """
    dates = df.index.get_level_values('date').to_timestamp()

    if earliest_train_date:
        earliest_train_datetime = datetime.strptime(earliest_train_date, date_format)
        df = df.loc[[earliest_train_datetime<=date for date in dates]]
        dates = df.index.get_level_values('date').to_timestamp()

    if last_test_date:
        last_test_datetime = datetime.strptime(last_test_date, date_format)
        df = df.loc[[date<last_test_datetime for date in dates]]
        dates = df.index.get_level_values('date').to_timestamp()

    split_datetime = datetime.strptime(split_date, date_format)
    train_dates = [date<split_datetime for date in dates]
    test_dates = np.invert(train_dates)

    train_df = df.loc[train_dates]
    test_df =  df.loc[test_dates]

    if verbose:
        print('================================')
        print('Training Dataset last rows')
        ICD.display(train_df.tail(3))
        print('================================')
        print('Test Dataset first rows')
        ICD.display(test_df.head(3))

    return train_df, test_df


def separate_X_and_y(df, target='sales'):
    y = df[target].values
    X = df.drop(columns=target)
    return X, y


def unstack_transformation(X, y, group_by='date', target='sales', levels=['store_nbr', 'family']):
    df = X.copy()
    df[target] = y
    y = df.loc[:, target].unstack(levels)
    X = df.drop(columns=[target]).groupby(by=group_by).first()
    return X, y


def create_sample_weights(X, target_date, weight=0.9):
    extra_weight_days = X.index.get_level_values('date') > target_date
    return np.array(list(map(lambda x: np.exp(-weight) if x == 0 else 1, extra_weight_days.astype('int'))))


def get_closed_stores(df, last_days=15):
    closed_stores_df = df.groupby(['family']).tail(last_days).groupby(['family'])['sales'].sum().reset_index()
    closed_stores_df = closed_stores_df[closed_stores_df['sales'] == 0].drop('sales', axis = 1)
    closed_stores_df = closed_stores_df[closed_stores_df['family'] != "SCHOOL AND OFFICE SUPPLIES"]
    closed_stores_df = closed_stores_df.set_index(['family'])
    closed_stores_df['isclosed'] = True
    return closed_stores_df


def apply_zero_forecasting(df, closed_stores_df):
    df = df.merge(closed_stores_df, how='left',  left_index=True, right_index=True)
    df.loc[df['isclosed']==True, 'sales']=0
    df.drop(columns=['isclosed'], inplace=True)
    return df


def timeit(function):
    def wrapper(*args, **kwargs):
        ts = time.perf_counter()
        func = function(*args, **kwargs)
        te = time.perf_counter ()
        run_time = time.perf_counter() - ts
        print(f'Run Time: {run_time} secs')
        return func
    return wrapper


def plot_feature_importance(feature_importances, feature_names):
    feats = {} # a dict to hold feature_name: feature_importance
    for feature, importance in zip(feature_names, feature_importances):
        feats[feature] = importance #add the name/value pair 

    importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
    importances.sort_values(by='Gini-importance').plot(kind='bar').show()


def plot_feature_elimination_score(min_features_to_select, grid_score):
    plt.figure()
    plt.xlabel('Number of features selected')
    plt.ylabel('Cross validation score')
    plt.plot(
        range(min_features_to_select, len(grid_score) + min_features_to_select),
        grid_score
    )
    plt.show()


def plot_predition_timeseries(x, y_actual, y_pred):
    df = pd.DataFrame()
    df['date'], df['actual'], df['prediction'] = x, y_actual, y_pred
    df = df.groupby(['date'], as_index=False).sum().reset_index()
    df['date'] = df['date'].apply(lambda row: row.to_timestamp())
    title = 'Actual vs Prediction'
    df.plot.line(x='date', y=['actual', 'prediction'], title=title).show()


def transform_prediction_to_df(X, y, prediction):
    return pd.DataFrame(pd.DataFrame(prediction,
                        index=X.index,
                        columns=y.columns).stack(['store_nbr'])).rename(columns={0: 'sales'})


def per_family_df_generator(train_df, test_df, verbose=False):
    families = sorted(train_df.index.get_level_values('family').unique())
    for _, family in zip(tqdm(range(len(families))), families):
        if verbose:
            print('='*40)
            print(f'Family: {family}')
        # Train DF   
        train_family_df = train_df[train_df.index.get_level_values('family')==family]
        train_family_df.index = train_family_df.index.droplevel('family')
            
        # Test DF 
        test_family_df = test_df[test_df.index.get_level_values('family')==family]
        test_family_df.index = test_family_df.index.droplevel('family')
        yield family, train_family_df, test_family_df

In [ ]:
# define estimators, metrics, ensemble and standalone models
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import BaggingRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression


class MultiOutBaggingExtraTreesRegressor(BaseEstimator, RegressorMixin):
    def __init__(self,
                extra_trees_n_estimators=200,
                bagging_n_estimators=10,
                log_target_transform=True,
                multioutput=True,
                n_jobs=-1,
                random_state=42):
        super().__init__()
        _r1 = ExtraTreesRegressor(n_estimators=extra_trees_n_estimators,
                                  n_jobs=n_jobs,
                                  random_state=random_state)
        _estimator = BaggingRegressor(base_estimator=_r1,
                                      n_estimators=bagging_n_estimators,
                                      bootstrap=True,
                                      n_jobs=n_jobs,
                                      random_state=random_state)
        if multioutput:
            _estimator = MultiOutputRegressor(_estimator)
        self.estimator_ = _estimator
        self.log_target_transform = log_target_transform
        self.feature_importances_ = None 
        self.bagging_n_estimators = bagging_n_estimators
        self.extra_trees_n_estimators = extra_trees_n_estimators
        self.multioutput = multioutput
        self.n_jobs = n_jobs
        self.random_state = random_state
    
    def fit(self, X, y, sample_weight=None):
        if self.log_target_transform:
            y = np.log1p(y)
        self.estimator_.fit(X, y, sample_weight)
        
        if self.multioutput:
            feature_importances_ = []
            for n_output_estimators in self.estimator_.estimators_:
                feature_importances_.append(np.mean([
                tree.feature_importances_ for tree in n_output_estimators.estimators_], axis=0))
            self.feature_importances_ = np.mean(feature_importances_, axis=0)   
        else:
            self.feature_importances_ = np.mean([
                tree.feature_importances_ for tree in self.estimator_.estimators_], axis=0)

        return self

    def predict(self, X, y=None):
        prediction = self.estimator_.predict(X)
        if self.log_target_transform:
            prediction = (np.exp(prediction)-1)
        return prediction.clip(0)

    def score(self, X, y):
        prediction = self.predict(X, y)
        return mean_squared_log_error(y, prediction, squared=False)

In [ ]:
# method to train one estimator per family and generate predictions
from sklearn.metrics import mean_squared_log_error

def family_prediction_generator(train_df, test_df, estimator_params={}, verbose=True):
    for family, train_family_df, test_family_df in per_family_df_generator(train_df, test_df, verbose=verbose):
        # generate multi-output (with unstack)
        X_train, y_train = separate_X_and_y(train_family_df)
        X_train, y_train = unstack_transformation(X_train, y_train, levels=['store_nbr'])
        X_test, y_test = separate_X_and_y(test_family_df)
        X_test, y_test = unstack_transformation(X_test, y_test, levels=['store_nbr'])

        # perform training
        no_multiple_outputs = ['BABY CARE', 'GROCERY II','HOME AND KITCHEN I','HOME AND KITCHEN II', 'SCHOOL AND OFFICE SUPPLIES']
        if family in no_multiple_outputs:
            estimator = MultiOutBaggingExtraTreesRegressor(**estimator_params, multioutput=False)
        else:
            estimator = MultiOutBaggingExtraTreesRegressor(**estimator_params)
            
        weights = create_sample_weights(X_train, '2017-07-01')    
        estimator.fit(X_train, y_train, sample_weight=weights)
        
        if plot_importances:
            feature_importances = estimator.feature_importances_
            feature_names = X_train.columns
            plot_feature_importance(feature_importances, feature_names)

        # generate predictions for training & test sets
        train_pred = estimator.predict(X_train)
        train_pred_df = transform_prediction_to_df(X_train, y_train, train_pred)
        train_pred_df['family'] = family
        train_pred_df = train_pred_df.set_index('family', append=True)

        test_pred = estimator.predict(X_test)
        test_pred_df = transform_prediction_to_df(X_test, y_test, test_pred)
        test_pred_df['family'] = family
        test_pred_df = test_pred_df.set_index('family', append=True)

        # score the predictions
        family_train_score = mean_squared_log_error(train_family_df['sales'].values,
                                                    train_pred_df['sales'].values,
                                                    squared=False)
        if verbose:
            print(f'RMSLE || TRAINING Score: {family_train_score}')
            
        if not any(np.isnan(test_family_df['sales'].values)):
            # Print score only in evaluation
            family_test_score = mean_squared_log_error(test_family_df['sales'].values,
                                                       test_pred_df['sales'].values,
                                                       squared=False)
            if verbose:
                print(f'RMSLE || TEST Score: {family_test_score}')  
        yield train_pred_df, test_pred_df

In [ ]:
# generate train and test dataframes
train_df, test_df = index_timeseries_train_test_split(df=feature_df,
                                                      date_index='date',
                                                      date_format='%Y-%m-%d',
                                                      earliest_train_date='2013-03-01',
                                                      split_date='2017-07-15', # '2016-08-01',
                                                      last_test_date='2017-08-01',  #'2016-09-15',
                                                      verbose=False)

In [ ]:
%%time
# train one estimator per family and generate predictions
estimator_params = {
                    'extra_trees_n_estimators': 5,
                    'bagging_n_estimators': 5,
    
    }

training_total_pred_df = pd.DataFrame()
test_total_pred_df = pd.DataFrame()
for train_pred_df, test_pred_df in family_prediction_generator(train_df, test_df, estimator_params):
    # retain predictions for later plotting
    training_total_pred_df = pd.concat([training_total_pred_df, train_pred_df])
    test_total_pred_df = pd.concat([test_total_pred_df, test_pred_df])

In [ ]:
print('===> Print All stores Training Prediction')
training_total_pred_df = training_total_pred_df.sort_values(['date', 'store_nbr', 'family'])
training_score = mean_squared_log_error(train_df['sales'].values, training_total_pred_df['sales'].values, squared=False)
plot_predition_timeseries(x=training_total_pred_df.index.get_level_values('date'), 
                            y_actual=train_df['sales'].values,
                            y_pred=training_total_pred_df['sales'].values)

print('===> Print All stores Test Prediction')
test_total_pred_df = test_total_pred_df.sort_values(['date', 'store_nbr', 'family'])
test_score = mean_squared_log_error(test_df['sales'].values, test_total_pred_df['sales'].values, squared=False)
plot_predition_timeseries(x=test_total_pred_df.index.get_level_values('date'), 
                            y_actual=test_df['sales'].values,
                            y_pred=test_total_pred_df['sales'].values)

print(f'RMSLE || TRAINING Score: {training_score} | TEST Score: {test_score}')

In [ ]:
%%time
# create the submission dataframe
train_df, test_df = index_timeseries_train_test_split(df=feature_df,
                                                      date_index='date',
                                                      date_format='%Y-%m-%d',
                                                      earliest_train_date='2013-03-01',
                                                      split_date='2017-08-16',
                                                      verbose=True)

# get stopped families for each store
closed_stores_df = get_closed_stores(train_df, last_days=12)


training_total_pred_df = pd.DataFrame()
test_total_pred_df = pd.DataFrame()
for train_pred_df, test_pred_df in family_prediction_generator(train_df, test_df):
    # retain prediction for submission
    training_total_pred_df = pd.concat([training_total_pred_df, train_pred_df])
    test_total_pred_df = pd.concat([test_total_pred_df, test_pred_df])

# Submission

In [ ]:
test_total_pred_df = test_total_pred_df.sort_values(['date', 'store_nbr', 'family'])

# Zero the sales for stopped families
test_total_pred_df= apply_zero_forecasting(test_total_pred_df, closed_stores_df)

ids = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')['id']
test_total_pred_df['id'] = ids.values

submit_df = test_total_pred_df[['id', 'sales']]

if len(submit_df) != 28512: # Minor check before submission :-)
    raise Exception(f'The rows ({len(submit_df)}) of the export are not correct')
else:
    current_dateTime = datetime.now()
    latest_filename = 'submission.csv'

    submit_df.to_csv(latest_filename, index=False)
    print('Submition created successfully!!')